# Numpy Calculation for Flash Problem

## Learning Objectives

After studying this notebook, completing the activities, and asking questions in class, you should be able to:
* Use numpy to solve the flash example problem from the [previous notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/05-System-of-Equations-Newton-Method.html").

In [32]:
# load libraries
import numpy as np
from scipy import optimize

## Motivation: Flash Problem Revisited

Recall the flash problem from [Chap 04-01 Notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/01-Modeling-Systems-of-Nonlinear-Equations.html") that we solved in the [last notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/05-System-of-Equations-Newton-Method.html#") using Newton's Method.

![flash](../../media/flash-system.png)

**Parameters (given)**:
* $F$ feed inlet flowrate, mol/time or kg/time
* $z_1$ composition of species 1 in feed, mol% or mass%
* $z_2$ composition of species 2 in feed, mol% or mass%
* $K_1$ partion coefficient for species 1, mol%/mol% or mass% / mass%
* $K_2$ partion coefficient for species 2, mol%/mol% or mass% / mass%

**Variables (unknown)**:
* $L$ liquid outlet flowrate, mol/time or kg/time
* $x_1$ composition of species 1 in liquid, mol% or mass%
* $x_2$ composition of species 2 in liquid, mol% or mass%
* $V$ vapor outlet flowrate, mol/time or kg/time
* $y_1$ composition of species 1 in vapor, mol% or mass%
* $y_2$ composition of species 2 in vapor, mol% or mass%

How to solve the flash problem and other multidimensional problem with $n$ unknown variables and $n$ nonlinear equations?

### System of Equations in Canonical Form

$$\mathbf{F}(\mathbf{x}) = \begin{pmatrix}
L + V - F\\
Vy_1 + L x_1 - F z_1 \\
V y_2 + L x_2 - F z_2 \\
y_1 - K_1 x_1 \\
y_2 - K_2 x_2 \\
y_1 + y_2 - x_1 - x_2
\end{pmatrix},$$

with $\mathbf{x} = (L, V, x_1, x_2, y_1, y_2).$

## Using ``numpy`` instead

`numpy` and `scipy` offer a few different implementations of Newton's method. However, we found these to be unreliable in the past. Instead, we recommend either using the Newton solver we put together in the [last notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/05-System-of-Equations-Newton-Method.html#") or Pyomo (future notebook).

However, we will show how this is done using `scipy.optimize.newton` in this notebook.

In [ ]:
# documentation for Newton's method in scipy
help(optimize.newton)

First run the code below to define the system of nonlinear equations given in the [previous notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/05-System-of-Equations-Newton-Method.html").

In [39]:
# nonlinear canonical system of equations
def my_f(x):
    ''' Nonlinear system of equations in conancial form F(x) = 0
    Copied from previous notebook.
    
    Arg:
        x: vector of variables
        
    Returns:
        r: residual, F(x)
    
    '''

    # Initialize residuals
    r = np.zeros(6)
    
    # given data
    F = 1.0
    z1 = 0.5
    z2 = 0.5
    K1 = 3
    K2 = 0.05
    
    # copy values from x to more meaningful names
    L = x[0]
    V = x[1]
    x1 = x[2]
    x2 = x[3]
    y1 = x[4]
    y2 = x[5]
    
    # equation 1: overall mass balance
    r[0] = V + L - F
    
    # equations 2 and 3: component mass balances
    r[1] = V*y1 + L*x1 - F*z1
    r[2] = V*y2 + L*x2 - F*z2
    
    # equation 4 and 5: equilibrium
    r[3] = y1 - K1*x1
    r[4] = y2 - K2*x2
    
    # equation 6: summation
    r[5] = (y1 + y2) - (x1 + x2)
    # This is known as the Rachford-Rice formulation for the summation constraint
    
    return r

Next, we need to define the initial guess as we did with our Newton System in the [previous notebook]("https://ndcbe.github.io/data-and-computing/notebooks/04-publish/05-System-of-Equations-Newton-Method.html").

In [40]:
# initial guess
x0 = np.array([0.5, 0.5, 0.55, 0.45, 0.65, 0.35])

Now we can run Newton's Method through `scipy.optimize.newton`.

In [41]:
xsln = optimize.newton(my_f,x0)
print("Solution using scipy:\n",xsln)

Solution using scipy:
 [ 5.00004967e-01  5.36467908e-01  1.09276056e+10  3.55480055e+00
 -5.38704523e+36  3.50020142e-01]
